The main solving big ridge regression problems

In [ ]:
workspace()

In [ ]:
using Plots

In [ ]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

In [ ]:
#Basic parameters
## Basic parameters
maxiter=1000;
max_time = 100;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [ ]:
dataset = "mnist.scale"   # mnist.scale  a9a  gisette_scale  rcv1_train.binary
prob = defineProb(dataset);
showDetails(dataset)

In [ ]:
norm(prob.A*prob.xsol - prob.b)

In [ ]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror,[]) 
OUTPUTS = [];

In [ ]:
# Testing Rademarcher Sketch
options.AUX = [1, 1]; # use sign flipping
method_name = "rademacher";# ROS
options.sketchsize =10;#Int(floor((prob.n)^(1/2)));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
options.AUX = [0 10]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 10; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
    
rho = Int(options.AUX[2]); # the density per row
s = options.sketchsize;
ind = sample(1:prob.n,rho*s,replace=false); # shuffle all indices
ind =reshape(ind, s,rho);
DATA = zeros(s,prob.n);

 function mapitf(A::Array{Float64}, ind, DATA)
 DATA = map( x-> sum(A[ind[x,:],:],1), 1:s)
 end

function loopitf(A::Array{Float64}, ind, DATA)
    for i =1:s
        DATA[i,:] = sum(A[ind[i,:],:],1);
    end
end

mapitf(prob.A,ind,DATA)

#loopitf(prob.A,ind,DATA)
#sum(prob.A[ind[1,:],:],1)
#DATA

In [ ]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =10; # Int(floor(log2(prob.n)));
options.skip_error_calculation =25;
@time  output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
# Testing countmin 
options.sketchsize =10;#floor((prob.n)^(1/2));
options.skip_error_calculation =25;
method_name = "countmin";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
# Testing CG 
options.skip_error_calculation =25;
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
# Testing CD 
method_name = "CD";# gradient descent
options.sketchsize = 10;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [ ]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

In [ ]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];